In [ ]:
import numpy as np
import torch
import torch.nn as nn
from ssm import SSM

L = 5
d_state = 16
A_scales_list = np.linspace(0, 1, 20).tolist()
n = 3
seeds = range(30)

cns = []
A_scales = []

for A_scale in A_scales_list:
    for i in seeds:
        torch.manual_seed(i)
        sd = SSM(num_input_channels=3, d_model=n, d_state=d_state, mup=True, use_kernel=True, A_scale=A_scale)
        sd.B = nn.Parameter(torch.ones(d_state))
        sd.C = nn.Parameter(torch.ones(d_state))

        cn = torch.linalg.cond(sd.kernel(L))
        cns.append(cn.item())
        A_scales.append(A_scale)
    # print(f"A_scale is {A_scale}; condition number is {cn}")


In [ ]:
import pandas as pd
import seaborn as sns

df = pd.DataFrame({'A_scale': A_scales, 'cond_num': cns})
sns.lineplot(data=df, x='A_scale', y='cond_num')

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from ssm import SSM
import math

L = 5
d_state = 1
A_scales_list = np.linspace(0, 1, 40).tolist()
n = 3
seeds = range(1)

cns = []
A_scales = []

def compute_imas_sq(a, L):
    ret = torch.zeros(L, L)
    ret += torch.diag(torch.ones(L)*(a**2 + 1))
    ret += torch.diag(torch.ones(L - 1)*-a, -1)
    ret += torch.diag(torch.ones(L - 1)*-a , 1)
    ret[0, 0] = 1
    return ret

for A_scale in A_scales_list:
    for i in seeds:
        torch.manual_seed(i)
        sd = SSM(num_input_channels=3, d_model=n, d_state=d_state, mup=True, use_kernel=True, A_scale=A_scale)
        sd.A = nn.Parameter(torch.ones(d_state)*A_scale)
        sd.B = nn.Parameter(torch.ones(d_state))
        sd.C = nn.Parameter(torch.ones(d_state))

        A_mat = sd.kernel._compute_A_mat(torch.ones(d_state)*A_scale, L)
        cn = torch.linalg.cond(A_mat)
        cns.append(cn.item())
        A_scales.append(A_scale)
    print(f"A_scale is {A_scale}; condition number is {cn}")


In [ ]:
import pandas as pd
import seaborn as sns

df = pd.DataFrame({'a': A_scales, 'cond_num': cns})
sns.lineplot(data=df, x='a', y='cond_num')

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from ssm import SSM
import math

L = 5
d_state = 1
A_scales_list = np.linspace(0, 1, 40).tolist()
n = 3
seeds = range(1)

cns = []
A_scales = []

def compute_imas_sq(a, L):
    ret = torch.zeros(L, L)
    ret += torch.diag(torch.ones(L)*(a**2 + 1))
    ret += torch.diag(torch.ones(L - 1)*-a, -1)
    ret += torch.diag(torch.ones(L - 1)*-a , 1)
    ret[0, 0] = 1
    return ret

for A_scale in A_scales_list:
    A_scale = 0.1
    for i in seeds:
        torch.manual_seed(i)
        sd = SSM(num_input_channels=3, d_model=n, d_state=d_state, mup=True, use_kernel=True, A_scale=A_scale)
        sd.A = nn.Parameter(torch.ones(d_state)*A_scale)
        sd.B = nn.Parameter(torch.ones(d_state))
        sd.C = nn.Parameter(torch.ones(d_state))

        A_mat = sd.kernel._compute_A_mat(torch.ones(d_state)*A_scale, L)

        predicted_sv = []
        for j in range(1, L):
            print(j)
            x = A_scale**2 + 1 - 2*A_scale*math.cos(j*math.pi / L)
            predicted_sv.append(math.sqrt(1/ x))
        print(predicted_sv)

        imas_sq = compute_imas_sq(A_scale, L)
        # imas_sq_m = imas_sq[:-1,:-1]
        # torch.set_printoptions(linewidth=100) 
        # print(torch.fft.ifft2(imas_sq))

        # print(torch.sqrt(1 /torch.svd(imas_sq)[1]))

        U, S, V = torch.svd(imas_sq)
        print(imas_sq)
        print(U.T @ torch.diag(S) @ V)

        print(A_mat)
        print(torch.svd(A_mat)[1])
        break
    break
    #     cn = torch.linalg.cond(A_mat)
    #     cns.append(cn.item())
    #     A_scales.append(A_scale)
    # print(f"A_scale is {A_scale}; condition number is {cn}")

In [ ]:
import torch

a = torch.tensor([2.2], requires_grad=True)

K = torch.eye(6)*a

# K.requires_grad = True

x = torch.arange(6, dtype=torch.float32)

# y = K @ x
y = torch.einsum("ij,j->i", K, x)
print(y)

loss = torch.sum(y**2)

loss.backward()
print(a.grad)

In [ ]:
import torch

for d_model in [6, 12, 18, 24, 36, 42, 48, 54]:

    a = torch.tensor([2.2], requires_grad=True)

    x = torch.rand(d_model, dtype=torch.float32) + 1
    # print(x)
    y = x*a
    # print(y)

    loss = torch.sum(y**2)
    print(a*2*torch.sum((x**2)))
    # print(loss)

    loss.backward()
    print(a.grad/ d_model)#(torch.sum(x**2))) #/ (d_model**3.1)) # Why is it 3?